In [49]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [50]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [51]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [52]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7733


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""4189""","""33 30 16 19 14""","""1416193033""","[14, 16, … 33]",14,16,19,30,33,"""14""","""16""","""19""","""30""","""33""",null,null,null,null,null,true,16,"""02_03_11_03""",-23
"""16/04/2025""","""4188""","""21 13 28 29 34""","""1321282934""","[13, 21, … 34]",13,21,28,29,34,"""13""","""21""","""28""","""29""","""34""","""14""","""16""","""19""","""30""","""33""",false,0,"""08_07_01_05""",15
"""15/04/2025""","""4187""","""29 15 23 33 10""","""1015232933""","[10, 15, … 33]",10,15,23,29,33,"""10""","""15""","""23""","""29""","""33""","""13""","""21""","""28""","""29""","""34""",false,0,"""05_08_06_04""",5
"""14/04/2025""","""4186""","""09 27 16 14 04""","""0409141627""","[4, 9, … 27]",4,9,14,16,27,"""04""","""09""","""14""","""16""","""27""","""10""","""15""","""23""","""29""","""33""",true,24,"""05_05_02_11""",15
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""","""04""","""09""","""14""","""16""","""27""",false,0,"""03_04_13_01""",-29
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""","""06""","""09""","""13""","""26""","""27""",false,0,"""07_02_04_07""",-6
"""11/04/2025""","""4183""","""09 10 08 01 12""","""0108091012""","[1, 8, … 12]",1,8,9,10,12,"""01""","""08""","""09""","""10""","""12""","""03""","""10""","""12""","""16""","""23""",true,24,"""07_01_01_02""",-5
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""08""","""09""","""10""","""12""",false,0,"""11_09_03_01""",8
"""09/04/2025""","""4181""","""29 21 20 28 04""","""0420212829""","[4, 20, … 29]",4,20,21,28,29,"""04""","""20""","""21""","""28""","""29""","""10""","""21""","""30""","""33""","""34""",true,16,"""16_01_07_01""",-33


In [53]:
dfRows.item(0, 'b2')

16

In [54]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [55]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='14'
df_viz FOR str_b1 = 14


post_str_b1,count,percent
str,u32,f64
"""02""",21,14.685315
"""03""",20,13.986014
"""01""",19,13.286713
"""06""",15,10.48951
"""05""",10,6.993007
…,…,…
"""12""",2,1.398601
"""16""",2,1.398601
"""21""",2,1.398601


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 14
Last 3 results last_results=['06', '04', '05']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""05/02/2025""","""4118""","""28 14 19 34 31""","""1419283134""","[14, 19, … 34]",14,19,28,31,34,"""14""","""19""","""28""","""31""","""34""","""06""","""11""","""14""","""20""","""25""",true,8,"""05_09_03_03""",16
"""03/09/2024""","""3963""","""15 33 14 20 29""","""1415202933""","[14, 15, … 33]",14,15,20,29,33,"""14""","""15""","""20""","""29""","""33""","""04""","""10""","""21""","""26""","""29""",false,0,"""01_05_09_04""",-9
"""30/07/2024""","""3928""","""25 17 30 23 14""","""1417232530""","[14, 17, … 30]",14,17,23,25,30,"""14""","""17""","""23""","""25""","""30""","""05""","""08""","""19""","""27""","""35""",false,0,"""03_06_02_05""",14
"""16/01/2024""","""3732""","""17 33 21 14 15""","""1415172133""","[14, 15, … 33]",14,15,17,21,33,"""14""","""15""","""17""","""21""","""33""","""04""","""09""","""11""","""15""","""20""",false,0,"""01_02_04_12""",5
"""18/10/2023""","""3642""","""17 14 18 26 34""","""1417182634""","[14, 17, … 34]",14,17,18,26,34,"""14""","""17""","""18""","""26""","""34""","""05""","""17""","""22""","""23""","""25""",true,8,"""03_01_08_08""",-16
"""15/10/2023""","""3639""","""23 21 34 24 14""","""1421232434""","[14, 21, … 34]",14,21,23,24,34,"""14""","""21""","""23""","""24""","""34""","""03""","""08""","""17""","""19""","""35""",false,0,"""07_02_01_10""",6
"""01/09/2023""","""3595""","""33 14 21 27 34""","""1421273334""","[14, 21, … 34]",14,21,27,33,34,"""14""","""21""","""27""","""33""","""34""","""03""","""04""","""05""","""06""","""24""",true,8,"""07_06_06_01""",-6
"""20/12/2022""","""3340""","""15 29 14 20 33""","""1415202933""","[14, 15, … 33]",14,15,20,29,33,"""14""","""15""","""20""","""29""","""33""","""06""","""08""","""11""","""22""","""27""",false,0,"""01_05_09_04""",-9
"""28/11/2022""","""3318""","""28 32 25 21 14""","""1421252832""","[14, 21, … 32]",14,21,25,28,32,"""14""","""21""","""25""","""28""","""32""","""06""","""13""","""23""","""32""","""33""",false,0,"""07_04_03_04""",0


df_jumps_map FOR str_b1 = 14


jumps_map,count
str,u32
"""08_01_04_02""",2
"""07_04_03_04""",2
"""01_05_09_04""",2
"""06_08_03_03""",2
"""07_02_04_06""",1
…,…
"""08_08_04_01""",1
"""01_04_03_04""",1
"""03_08_04_04""",1


df_unijump FOR str_b1 = 14


unijump,count
i64,u32
-7,7
3,6
-6,6
-1,6
-11,5
…,…
18,1
-39,1
1,1


In [56]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='16'
df_viz FOR str_b2 = 16


post_str_b2,count,percent
str,u32,f64
"""07""",26,7.902736
"""13""",20,6.079027
"""15""",20,6.079027
"""12""",19,5.775076
"""16""",19,5.775076
…,…,…
"""25""",4,1.215805
"""24""",3,0.911854
"""28""",2,0.607903


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 16
Last 3 results last_results=['08', '12', '04']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""12""","""13""","""25""","""33""",false,0,"""09_06_04_08""",5
"""16/03/2025""","""4157""","""26 10 21 16 30""","""1016212630""","[10, 16, … 30]",10,16,21,26,30,"""10""","""16""","""21""","""26""","""30""","""04""","""08""","""20""","""24""","""30""",true,8,"""06_05_05_04""",-2
"""08/03/2025""","""4149""","""02 35 30 32 16""","""0216303235""","[2, 16, … 35]",2,16,30,32,35,"""02""","""16""","""30""","""32""","""35""","""02""","""04""","""15""","""16""","""31""",true,8,"""14_14_02_03""",25
"""05/03/2025""","""4146""","""22 18 16 06 20""","""0616182022""","[6, 16, … 22]",6,16,18,20,22,"""06""","""16""","""18""","""20""","""22""","""13""","""17""","""21""","""30""","""35""",true,28,"""10_02_02_02""",-8
"""13/01/2025""","""4095""","""08 28 18 22 16""","""0816182228""","[8, 16, … 28]",8,16,18,22,28,"""08""","""16""","""18""","""22""","""28""","""08""","""18""","""22""","""23""","""35""",true,20,"""08_02_04_06""",-8
"""11/01/2025""","""4093""","""22 02 25 27 16""","""0216222527""","[2, 16, … 27]",2,16,22,25,27,"""02""","""16""","""22""","""25""","""27""","""04""","""07""","""16""","""18""","""23""",true,16,"""14_06_03_02""",-3
"""07/01/2025""","""4089""","""26 35 28 16 13""","""1316262835""","[13, 16, … 35]",13,16,26,28,35,"""13""","""16""","""26""","""28""","""35""","""02""","""08""","""17""","""24""","""27""",false,0,"""03_10_02_07""",28
"""02/01/2025""","""4084""","""27 35 08 16 26""","""0816262735""","[8, 16, … 35]",8,16,26,27,35,"""08""","""16""","""26""","""27""","""35""","""04""","""13""","""15""","""18""","""34""",false,0,"""08_10_01_08""",27
"""19/10/2024""","""4009""","""19 31 17 13 16""","""1316171931""","[13, 16, … 31]",13,16,17,19,31,"""13""","""16""","""17""","""19""","""31""","""18""","""23""","""30""","""33""","""35""",true,17,"""03_01_02_12""",6


df_jumps_map FOR str_b2 = 16


jumps_map,count
str,u32
"""13_04_04_08""",2
"""01_04_07_04""",2
"""09_01_01_16""",1
"""12_06_03_09""",1
"""15_10_03_02""",1
…,…
"""15_03_10_01""",1
"""05_05_04_08""",1
"""06_04_10_05""",1


df_unijump FOR str_b2 = 16


unijump,count
i64,u32
-18,11
3,10
-3,9
-8,9
11,9
…,…
-20,1
20,1
-23,1


In [57]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='19'
df_viz FOR str_b3 = 19


post_str_b3,count,percent
str,u32,f64
"""19""",31,7.616708
"""20""",29,7.125307
"""15""",27,6.633907
"""21""",26,6.388206
"""22""",23,5.651106
…,…,…
"""05""",4,0.982801
"""06""",4,0.982801
"""31""",3,0.737101


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 19
Last 3 results last_results=['10', '12', '20']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""22/03/2025""","""4163""","""19 32 17 10 22""","""1017192232""","[10, 17, … 32]",10,17,19,22,32,"""10""","""17""","""19""","""22""","""32""","""04""","""08""","""10""","""13""","""15""",false,0,"""07_02_03_10""",0
"""10/03/2025""","""4151""","""29 06 26 15 19""","""0615192629""","[6, 15, … 29]",6,15,19,26,29,"""06""","""15""","""19""","""26""","""29""","""02""","""11""","""12""","""16""","""32""",true,16,"""09_04_07_03""",-15
"""11/02/2025""","""4124""","""08 32 28 19 17""","""0817192832""","[8, 17, … 32]",8,17,19,28,32,"""08""","""17""","""19""","""28""","""32""","""12""","""13""","""20""","""31""","""33""",false,0,"""09_02_09_04""",-26
"""31/01/2025""","""4113""","""19 28 32 05 04""","""0405192832""","[4, 5, … 32]",4,5,19,28,32,"""04""","""05""","""19""","""28""","""32""","""04""","""05""","""08""","""16""","""35""",false,0,"""01_14_09_04""",18
"""20/01/2025""","""4102""","""19 03 34 25 04""","""0304192534""","[3, 4, … 34]",3,4,19,25,34,"""03""","""04""","""19""","""25""","""34""","""21""","""23""","""24""","""30""","""34""",false,0,"""01_15_06_09""",35
"""04/01/2025""","""4086""","""06 10 29 32 19""","""0610192932""","[6, 10, … 32]",6,10,19,29,32,"""06""","""10""","""19""","""29""","""32""","""07""","""19""","""20""","""23""","""28""",false,0,"""04_09_10_03""",-4
"""22/12/2024""","""4073""","""19 20 35 07 05""","""0507192035""","[5, 7, … 35]",5,7,19,20,35,"""05""","""07""","""19""","""20""","""35""","""06""","""09""","""22""","""24""","""32""",false,0,"""02_12_01_15""",46
"""03/12/2024""","""4054""","""19 31 06 25 15""","""0615192531""","[6, 15, … 31]",6,15,19,25,31,"""06""","""15""","""19""","""25""","""31""","""05""","""06""","""11""","""16""","""23""",true,8,"""09_04_06_06""",-9
"""18/11/2024""","""4039""","""01 07 19 23 24""","""0107192324""","[1, 7, … 24]",1,7,19,23,24,"""01""","""07""","""19""","""23""","""24""","""07""","""10""","""17""","""21""","""24""",false,0,"""06_12_04_01""",19


df_jumps_map FOR str_b3 = 19


jumps_map,count
str,u32
"""01_06_14_02""",3
"""05_06_09_02""",2
"""01_01_05_10""",2
"""03_07_15_01""",2
"""09_06_04_09""",2
…,…
"""04_12_04_02""",1
"""01_17_06_02""",1
"""06_10_02_04""",1


df_unijump FOR str_b3 = 19


unijump,count
i64,u32
3,13
6,12
-6,12
1,12
-4,11
…,…
-37,1
38,1
-42,1


In [58]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='30'
df_viz FOR str_b4 = 30


post_str_b4,count,percent
str,u32,f64
"""27""",34,8.252427
"""22""",30,7.281553
"""26""",28,6.796117
"""23""",27,6.553398
"""28""",26,6.31068
…,…,…
"""34""",3,0.728155
"""12""",2,0.485437
"""09""",1,0.242718


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 30
Last 3 results last_results=['26', '33', '30']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""06/03/2025""","""4147""","""30 17 13 21 35""","""1317213035""","[13, 17, … 35]",13,17,21,30,35,"""13""","""17""","""21""","""30""","""35""","""03""","""07""","""24""","""26""","""30""",true,8,"""04_04_09_05""",-14
"""21/02/2025""","""4134""","""32 28 22 10 30""","""1022283032""","[10, 22, … 32]",10,22,28,30,32,"""10""","""22""","""28""","""30""","""32""","""03""","""15""","""24""","""33""","""34""",true,12,"""12_06_02_02""",2
"""20/02/2025""","""4133""","""34 15 26 04 30""","""0415263034""","[4, 15, … 34]",4,15,26,30,34,"""04""","""15""","""26""","""30""","""34""","""10""","""22""","""28""","""30""","""32""",true,8,"""11_11_04_04""",14
"""13/02/2025""","""4126""","""17 30 21 33 15""","""1517213033""","[15, 17, … 33]",15,17,21,30,33,"""15""","""17""","""21""","""30""","""33""","""09""","""14""","""15""","""26""","""28""",false,0,"""02_04_09_03""",-14
"""21/01/2025""","""4103""","""23 34 30 24 21""","""2123243034""","[21, 23, … 34]",21,23,24,30,34,"""21""","""23""","""24""","""30""","""34""","""06""","""12""","""17""","""18""","""35""",false,0,"""02_01_06_04""",-13
"""15/01/2025""","""4097""","""30 21 25 24 32""","""2124253032""","[21, 24, … 32]",21,24,25,30,32,"""21""","""24""","""25""","""30""","""32""","""02""","""09""","""17""","""20""","""21""",false,0,"""03_01_05_02""",-13
"""06/01/2025""","""4088""","""01 11 34 05 30""","""0105113034""","[1, 5, … 34]",1,5,11,30,34,"""01""","""05""","""11""","""30""","""34""","""13""","""16""","""26""","""28""","""35""",true,16,"""04_06_19_04""",-39
"""29/12/2024""","""4080""","""15 30 34 01 07""","""0107153034""","[1, 7, … 34]",1,7,15,30,34,"""01""","""07""","""15""","""30""","""34""","""10""","""12""","""17""","""31""","""32""",true,16,"""06_08_15_04""",-23
"""26/12/2024""","""4077""","""25 10 30 32 28""","""1025283032""","[10, 25, … 32]",10,25,28,30,32,"""10""","""25""","""28""","""30""","""32""","""12""","""18""","""20""","""23""","""28""",true,8,"""15_03_02_02""",-10


df_jumps_map FOR str_b4 = 30


jumps_map,count
str,u32
"""10_07_02_05""",2
"""04_02_04_03""",2
"""05_05_13_05""",2
"""22_03_03_02""",2
"""08_19_01_03""",2
…,…
"""07_10_02_01""",1
"""10_08_02_05""",1
"""06_08_13_05""",1


df_unijump FOR str_b4 = 30


unijump,count
i64,u32
-22,10
-1,10
-9,10
-10,9
2,9
…,…
-51,1
-42,1
-57,1


In [59]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b5', post_col='post_str_blast')

Setting test_value='33'
df_viz FOR str_b5 = 33


post_str_blast,count,percent
str,u32,f64
"""35""",121,14.020857
"""33""",98,11.355736
"""34""",97,11.239861
"""32""",83,9.617613
"""31""",77,8.922364
…,…,…
"""16""",6,0.695249
"""15""",5,0.579374
"""14""",3,0.347625


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 33
Last 3 results last_results=['26', '34', '32']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""15/04/2025""","""4187""","""29 15 23 33 10""","""1015232933""","[10, 15, … 33]",10,15,23,29,33,"""10""","""15""","""23""","""29""","""33""","""13""","""21""","""28""","""29""","""34""",false,0,"""05_08_06_04""",5
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""14""","""17""","""24""","""26""",false,0,"""04_01_12_08""",-29
"""21/03/2025""","""4162""","""32 33 01 17 26""","""0117263233""","[1, 17, … 33]",1,17,26,32,33,"""01""","""17""","""26""","""32""","""33""","""10""","""17""","""19""","""22""","""32""",false,0,"""16_09_06_01""",-6
"""28/02/2025""","""4141""","""33 04 28 22 07""","""0407222833""","[4, 7, … 33]",4,7,22,28,33,"""04""","""07""","""22""","""28""","""33""","""02""","""05""","""11""","""25""","""32""",true,16,"""03_15_06_05""",29
"""17/02/2025""","""4130""","""12 33 22 19 20""","""1219202233""","[12, 19, … 33]",12,19,20,22,33,"""12""","""19""","""20""","""22""","""33""","""01""","""05""","""13""","""20""","""26""",false,0,"""07_01_02_11""",1
"""16/02/2025""","""4129""","""33 07 12 04 32""","""0407123233""","[4, 7, … 33]",4,7,12,32,33,"""04""","""07""","""12""","""32""","""33""","""12""","""19""","""20""","""22""","""33""",false,0,"""03_05_20_01""",-47
"""13/02/2025""","""4126""","""17 30 21 33 15""","""1517213033""","[15, 17, … 33]",15,17,21,30,33,"""15""","""17""","""21""","""30""","""33""","""09""","""14""","""15""","""26""","""28""",false,0,"""02_04_09_03""",-14
"""12/02/2025""","""4125""","""31 33 20 12 13""","""1213203133""","[12, 13, … 33]",12,13,20,31,33,"""12""","""13""","""20""","""31""","""33""","""15""","""17""","""21""","""30""","""33""",false,0,"""01_07_11_02""",-11
"""03/02/2025""","""4116""","""17 31 32 33 19""","""1719313233""","[17, 19, … 33]",17,19,31,32,33,"""17""","""19""","""31""","""32""","""33""","""06""","""13""","""23""","""34""","""35""",true,8,"""02_12_01_01""",32


df_jumps_map FOR str_b5 = 33


jumps_map,count
str,u32
"""05_03_09_08""",2
"""04_02_04_03""",2
"""12_10_04_01""",2
"""06_08_14_03""",2
"""01_05_09_04""",2
…,…
"""01_09_12_09""",1
"""07_02_06_07""",1
"""06_06_01_06""",1


df_unijump FOR str_b5 = 33


unijump,count
i64,u32
0,22
5,21
2,20
-1,20
10,20
…,…
-58,1
60,1
59,1


In [64]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [65]:
parquet_file = './parquet_gn_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 5),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [ ]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0105112030""","[1, 5, … 30]",1,5,11,20,30,"""01""","""05""","""11""","""20""","""30""","""01""","""29""",false,0,"""04_06_09_10""",-3
"""0105112031""","[1, 5, … 31]",1,5,11,20,31,"""01""","""05""","""11""","""20""","""31""","""01""","""30""",false,0,"""04_06_09_11""",-2
"""0105112033""","[1, 5, … 33]",1,5,11,20,33,"""01""","""05""","""11""","""20""","""33""","""01""","""32""",false,0,"""04_06_09_13""",0
"""0105112035""","[1, 5, … 35]",1,5,11,20,35,"""01""","""05""","""11""","""20""","""35""","""01""","""34""",false,0,"""04_06_09_15""",2
"""0105112130""","[1, 5, … 30]",1,5,11,21,30,"""01""","""05""","""11""","""21""","""30""","""01""","""29""",false,0,"""04_06_10_09""",-7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1120272933""","[11, 20, … 33]",11,20,27,29,33,"""11""","""20""","""27""","""29""","""33""","""11""","""32""",false,0,"""09_07_02_04""",10
"""1120272935""","[11, 20, … 35]",11,20,27,29,35,"""11""","""20""","""27""","""29""","""35""","""11""","""34""",false,0,"""09_07_02_06""",12
"""1120273133""","[11, 20, … 33]",11,20,27,31,33,"""11""","""20""","""27""","""31""","""33""","""11""","""32""",false,0,"""09_07_04_02""",2


In [63]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0305172528""","[3, 5, … 28]",3,5,17,25,28,"""03""","""05""","""17""","""25""","""28""","""03""","""27""",false,0,"""02_12_08_03""",13
